This notebook will detail the process of designing a TensorFlow text analysis program, on a BBC news Fake News Dataset and a Reddit MultiLabel Sentiment Analysis.

The Fake News dataset will be done using pretty standard NLP Techniques and the Sentiment Analysis One will leverage Transfer Learning to achieve the task(On BERT model)

Both Datasets come from Kaggle.

# Load Dependencies

In [ ]:
%%capture
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import nltk
import copy
from bs4 import BeautifulSoup
import tqdm.notebook as tqdm
import random
import math
nltk.download("punkt")
nltk.download('stopwords')
!pip install kaggle

In [ ]:
%%capture
!mkdir /root/.kaggle/
!cp -f ./kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
%%capture
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset
!unzip fake-and-real-news-dataset.zip
!rm -f fake-and-real-news-dataset.zip

# Load in the Dataset

In [ ]:
fake_news_pd = pd.read_csv("Fake.csv")
real_news_pd = pd.read_csv("True.csv")

In [ ]:
fake_news_corpus = fake_news_pd['title']
real_news_corpus = real_news_pd['title']

In [ ]:
def process_corpus(news):
  '''
  Processes and prepares a News Corpus, stripping stopwords, lower casing everything, and stemming words
  It also word tokenizes every sentence
  '''
  snowBallStemmer = nltk.SnowballStemmer('english')
  tokenized = [nltk.word_tokenize(sent) for sent in news]
  stopwords = nltk.corpus.stopwords.words("english")
  corpus = []
  for sent_idx in tqdm.tqdm(range(len(tokenized))):
    sent = tokenized[sent_idx]
    sent_processed = []
    for word_idx in range(len(sent)):
      if sent[word_idx] not in stopwords:
        sent_processed += [snowBallStemmer.stem(str.lower(sent[word_idx]))]
    corpus += [sent_processed]
  return corpus


In [ ]:
fake_news_corpus = process_corpus(fake_news_corpus)
real_news_corpus = process_corpus(real_news_corpus)

In [ ]:
fake_news_test = fake_news_corpus[0:16]
fake_news_corpus = fake_news_corpus[16:]
real_news_test = real_news_corpus[0:16]
real_news_corpus = real_news_corpus[16:]

In [ ]:
class FakeNewsDataset(keras.utils.Sequence):
  def __init__(self, fake_news, real_news, batch_size):
    self.fake_news = fake_news
    self.real_news = real_news
    self.fake_news_classes = {str(fn): 0 for fn in self.fake_news}
    self.real_news_classes = {str(rn): 1 for rn in self.real_news} # Real News: 1, Fake News: 0
    self.corpus_classes = {**self.fake_news_classes, **self.real_news_classes}
    self.corpus = self.fake_news + self.real_news

    random.shuffle(self.corpus)
    self.batch_size = batch_size
    self.cur_idx = 0
  def __len__(self):
    return len(self.corpus) // self.batch_size
  def __getitem__(self, idx):
    article = self.corpus[self.cur_idx * self.batch_size: (self.cur_idx + 1) * self.batch_size]
    self.cur_idx += 1
    if self.cur_idx >= self.__len__():
      self.cur_idx = 0
    class_idx = []
    for x in article:
      class_idx += [self.corpus_classes[str(x)]]
    return article, np.array(class_idx, dtype = np.float)

In [ ]:
dataset = FakeNewsDataset(fake_news_corpus, real_news_corpus, batch_size=16)

# Model: Bidirectional Stacked LSTM with Word and Character Embeddings 
Attention will be added in a future notebook, when Pure TensorFlow is used and attention is made from scratch.

In [ ]:
class CharEmbeddings(keras.layers.Layer):
  '''
  Character Embeddings, Made From Scratch to support both sentence and word padding(its also faster.)
  '''
  def __init__(self, corpus):
    super().__init__()
    self.unique_chars = list(self._compute_unique_characters(corpus))
    self.chars_to_idx = {self.unique_chars[i]: i + 2 for i in range(len(self.unique_chars))}
    self.chars_to_idx['<PAD>'] = 0
    self.chars_to_idx['<UNK>'] = 1

    self.idx_to_chars = {i + 2: self.unique_chars[i] for i in range(len(self.unique_chars))}
    self.idx_to_chars[0] = '<PAD>'
    self.idx_to_chars[1] = '<UNK>'
    self.char_embeddings = keras.layers.Embedding(len(self.unique_chars) + 2, 128)
  def _compute_unique_characters(self, corpus):
    characters = [character for article in tqdm.tqdm(corpus) for word in article for character in word]
    return set(characters)
  def _compute_max_sentence_length(self, sentences):
    '''
    Computes the length of the longest sentence in a list of tokenized items
    '''
    max_length = 0
    for article in sentences:
      max_length = max(max_length, len(article))
    return max_length
  def _compute_max_word_length(self, sentences):
    '''
    Computes the longest word length in the corpus
    '''
    max_length = 0
    for article in sentences:
      for word in article:
        max_length = max(max_length, len(word))
    return max_length
  def pad_words(self, x, sent_length, max_word_length = None):
    '''
    Pads words up to max_words and pads sentences up to sent_length.
    Note: Cut off sent_length and max length
    '''
    if not max_word_length:
      max_word_length = self._compute_max_word_length(x)
    article_padded = []
    for article in x:
      sent_padded = [self.chars_to_idx['<PAD>']] * sent_length
      for word_idx in range(sent_length):
        word_padded = [self.chars_to_idx['<PAD>']] * max_word_length
        if word_idx < len(article):
          for char_idx in range(len(article[word_idx])):
            if char_idx < max_word_length:
              if article[word_idx][char_idx] in self.chars_to_idx:
                word_padded[char_idx] = self.chars_to_idx[article[word_idx][char_idx]]
              else:
                # <UNK> CHAR
                word_padded[char_idx] = self.chars_to_idx['<UNK>']
        sent_padded[word_idx] = word_padded
      article_padded += [sent_padded]
    return article_padded
           
  def _get_embeddings(self, x):
    # x: One Sentence()
    # Tokenize the Character Embeddings
    embeddings = self.char_embeddings(padded_tokens) # (Num words, num chars, 256)
    return np.array(embeddings)

  def _split_into_characters(self, x):
    vals = []
    for sent in x:
      sent_words = []
      for word in sent:
        sent_words += [list(word)]
      vals += [sent_words]
    return vals

  def call(self, x, longest_sent = None, longest_word = None):
    # x: List of Sentences
    tokenized_chars = self._split_into_characters(x)
    if not longest_sent:
      longest_sent = self._compute_max_sentence_length(x)
    idx_tokenized_chars = tf.convert_to_tensor(self.pad_words(tokenized_chars, longest_sent, max_word_length = longest_word))
    tensor = np.array(idx_tokenized_chars) # (B, W, C)
    B, _, _ = tensor.shape
    embeddings = []
    for b in range(B):
      words = tensor[b, :, :] # (W, C)
      word_embeddings = self.char_embeddings(words)
      embeddings += [word_embeddings]
    return np.stack(embeddings) # (B, W, C, 256)

In [ ]:
class WordEmbeddings(tf.keras.layers.Layer):
  '''
  Prepares the Embedding Layer given some corpus

  Hand Prepared using nltk, for consistency with the character version
  '''
  def __init__(self, corpus):
    super().__init__()
    self.unique_words = list(self._compute_unique_words(corpus))
    self.word_2_idx = {self.unique_words[i]: i + 2 for i in range(len(self.unique_words))}
    self.word_2_idx['<PAD>'] = 0
    self.word_2_idx['<UNK>'] = 1
    self.idx_2_word = {i + 2: self.unique_words[i] for i in range(len(self.unique_words))}
    self.idx_2_word[0] = '<PAD>'
    self.idx_2_word[1] = '<UNK>'
    self.embeddings = keras.layers.Embedding(len(self.unique_words) + 2, 128)
  def _compute_unique_words(self, corpus):
    '''
    Computes the set of unique words in this corpus
    '''
    words = [word for text in tqdm.tqdm(corpus) for word in text]
    return set(words)
  def _compute_max_sentence(self, x):
    '''
    Computes the Length of the longest sentence in the corpus
    '''
    max_length = 0
    for article in x:
      max_length = max(len(article), max_length)
    return max_length
  def _pad_sents(self, x, max_sent):
    '''
    Pads and tokenizes each word in th corpus
    '''
    tokenized_text = []
    for sent in x:
      padded_article = [self.word_2_idx['<PAD>']] * max_sent
      for word_idx in range(len(sent)):
        if word_idx >= max_sent:
          break # Cut off Sentence
        if sent[word_idx] in self.word_2_idx:
          padded_article[word_idx] = self.word_2_idx[sent[word_idx]]
        else:
          # <UNK> Token
          padded_article[word_idx] = self.word_2_idx['<UNK>']
      tokenized_text += [padded_article]
    return tokenized_text
  def call(self, x, max_sent_length = None):
    if not max_sent_length:
      max_sent_length = self._compute_max_sentence(x)
    idx_sentences = tf.convert_to_tensor(self._pad_sents(x, max_sent_length))
    tensor = np.array(idx_sentences) 
    #print(tensor)
    return self.embeddings(tensor)

  

In [ ]:
class NLPEmbeddings(keras.layers.Layer):
  def __init__(self, corpus):
    super().__init__()
    self.word_embeddings = WordEmbeddings(corpus)
    self.char_embeddings = CharEmbeddings(corpus)
    self.Conv1 = keras.layers.Conv1D(128, 7, padding = 'same', activation = 'relu')
    self.Highway_proj = keras.layers.Dense(256, activation = 'relu')
    self.Highway_gate = keras.layers.Dense(256, activation = 'sigmoid')
  def call(self, x, longest_sent = None, longest_word = None):
    character_embeddings = self.char_embeddings(x, longest_sent = longest_sent, longest_word = longest_word) # (B, W, C, 256)
    word_embeddings = self.word_embeddings(x, max_sent_length = longest_sent) # (B, W, 256)
    # Reshape the character embeddings
    B, W, C, _ = character_embeddings.shape
    character_embeddings = np.reshape(character_embeddings, (B * W, C, 128))
    
    # Run Convolution on the Characters 
    convolution = self.Conv1(character_embeddings) # (B * W, C, 256)
    # Max Pool over features
    max_pooled = tf.math.reduce_max(convolution, axis = 1) # (B * W, 256)
    max_pooled = np.reshape(max_pooled, (B, W, 128))
    # Concatenate the features
    word_embedding = np.concatenate([word_embeddings, max_pooled], axis = -1) # (B, W, 512)
    
    proj = self.Highway_proj(word_embedding)
    gate = self.Highway_gate(word_embedding)

    highway = proj * gate + (1 - gate) * word_embedding
    return word_embedding
  

In [ ]:
class LSTM(keras.layers.Layer):
  def __init__(self):
    super().__init__()
    self.LSTMLayer1 = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences = True, dropout = 0.1, recurrent_dropout = 0.1))
    self.LSTMLayer2 = keras.layers.Bidirectional(keras.layers.LSTM(512))  
  def call(self, x):
    LSTMLayer1 = self.LSTMLayer1(x)
    LSTMLayer2 = self.LSTMLayer2(LSTMLayer1)
    return LSTMLayer2

In [ ]:
class BNBlock(keras.layers.Layer):
  def __init__(self, out_features, kernel_size, stride):
    super().__init__()
    self.out_features = out_features
    self.kernel_size = kernel_size
    self.stride = stride
    self.conv = keras.layers.Conv1D(self.out_features, self.kernel_size, strides = self.stride, padding = 'same', activation = 'relu')
  def call(self, x):
    return self.conv(x)

In [ ]:
class CNN(keras.layers.Layer):
  def __init__(self):
    # Expected Input: (B, 30, 512)
    super().__init__()
    self.Conv1 = BNBlock(256, 7, 1)
    self.Conv2 = BNBlock(512, 7, 1)
    self.Conv3 = BNBlock(1024, 7, 1)
    self.Conv4 = BNBlock(1024, 7, 1)
  def call(self, x):
    return self.Conv4(self.Conv3(self.Conv2(self.Conv1(x))))

In [ ]:
class LSTMModel(keras.Model):
  def __init__(self, corpus):
    super().__init__()
    self.embeddings = NLPEmbeddings(corpus)
    self.CNN = CNN()
    self.LSTM = LSTM()
    self.Dense = keras.layers.Dense(1)
  def call(self, x, max_sent_length, max_word_length):
    word_embeddings = self.embeddings.call(x, longest_sent = max_sent_length, longest_word = max_word_length)
    convolved_embeddings = self.CNN(word_embeddings)
    encoded_embeddings = self.LSTM(convolved_embeddings)
    #encoded_embeddings = tf.reduce_max(convolved_embeddings, axis = 1)
    return self.Dense(encoded_embeddings)

In [ ]:
model = LSTMModel(dataset.corpus)

In [ ]:
# Test set
test_dataset = FakeNewsDataset(fake_news_test, real_news_test, batch_size = 32)

In [ ]:
def step(x, y):
  criterion = tf.keras.losses.BinaryCrossentropy(from_logits = True)
  optim = tf.keras.optimizers.Adam(learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(1e-4, 2, 0.95))
  with tf.GradientTape() as tape:
    pred = tf.squeeze(model(x, max_sent_length = 30, max_word_length = 20, training = True))    
    loss = criterion(y, pred)
  grads = tape.gradient(loss, model.trainable_variables)
  optim.apply_gradients(zip(grads, model.trainable_variables))
  return loss 

In [ ]:
def test():
  with tf.device("GPU:0"):
      for x, y in test_dataset:
        pred = tf.squeeze(tf.sigmoid(model(x, max_sent_length = 30, max_word_length = 20, training = False))).numpy()
        one = pred >= 0.5 # Round Values to ones and zeros
        pred[:] = 0
        B = pred.shape
        pred[one] = 1
        print(pred)
        vals = pred == y
        summed = np.sum(vals)
        return summed / B 


In [ ]:
def train(num_epochs, print_every = 256):
  for EPOCH in range(num_epochs):
    total_loss = 0
    count = 0
    for x, y in tqdm.tqdm(dataset):
      total_loss += step(x, y).numpy().item()
      count += 1
      if count == print_every:
        break
    # Get test accuracy
    accuracy = test()
    print(f"EPOCH: {EPOCH}, total_loss: {total_loss / count}, accuracy: {accuracy}")

In [ ]:
with tf.device("GPU:0"):
  train(50, print_every = 64)

In [ ]:
model.save_weights("./model/model")

In [ ]:
model.load_weights("./model")

# Part 2: Sentimental Analysis With Transfer Learning
We will be abstracting away BERT and Transfer Learning using BERT.

The Sentimental Analysis dataset will be the IMDB Dataset, on multi sentiment labels.

In [ ]:
# Load in the Dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip
!rm -f imdb-dataset-of-50k-movie-reviews.zip

 66% 17.0M/25.7M [00:00<00:00, 54.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 85.9MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
imdb_csv = pd.read_csv("./IMDB Dataset.csv")

Process and Load in dataset

In [ ]:
sentiments = [1 if imdb_csv['sentiment'][idx] == 'positive' else 0 for idx in tqdm.tqdm(range(len(imdb_csv)))]
reviews = [review for review in imdb_csv['review']]

In [ ]:
sentiments_train = sentiments[:10000]
reviews_train = reviews[:10000]

sentiments_test = sentiments[10000: 10016]
reviews_test = reviews[10000: 10016]

sentiments_val = sentiments[10016: 10032]
reviews_val = reviews[10016: 10032]

HuggingFace Transformers: BERT 






In [ ]:
%%capture 
!pip install transformers
import transformers

Simple Tokenizer for Testing Purposes

In [ ]:
class DistilledBERTTokenizer(keras.Model):
  def __init__(self):
    super().__init__()
    self.tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
  def call(self, x):
    tokenized = self.tokenizer(x, truncation = True, return_attention_mask = True, padding = "max_length", return_tensors = "tf")
    return tokenized

In [ ]:
class TransferLearnBERT(keras.Model):
  def __init__(self, num_classes):
    super().__init__()
    self.num_classes = num_classes
    self.Tokenizer = DistilledBERTTokenizer()
    self.DBERT = transformers.TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    self.Dense = keras.layers.Dense(self.num_classes)
  def call(self, x):
    tokenized = self.Tokenizer(x)
    predicted = self.DBERT(**tokenized)
    hidden_state = predicted[0] # (B, L, C) 
    avg_hidden_state = tf.reduce_mean(hidden_state, axis = 1) # (B, C)
    return self.Dense(avg_hidden_state)


In [ ]:
model = TransferLearnBERT(1)

Training Loop

In [ ]:
def training_fn(corpus, GroundTruths, validation_corpus, validation_GT, NUM_EPOCHS, display_every = 64):
  optim = tf.optimizers.Adam(learning_rate= 1e-4)
  batch_size = 4
  cur_idx = 0
  np_validation_GT = np.array(validation_GT)
  for EPOCH in range(NUM_EPOCHS):
    total_loss = 0
    count = 0
    for idx in tqdm.tqdm(range(display_every)):
      paragraph = corpus[cur_idx * batch_size: (cur_idx + 1) * batch_size]
      GT = np.array(GroundTruths[cur_idx * batch_size: (cur_idx + 1) * batch_size])
      cur_idx += 1
      if cur_idx >= len(corpus) // batch_size:
        cur_idx = 0
      with tf.GradientTape() as tape:
        pred = model(paragraph, training = True)
        loss = tf.losses.binary_crossentropy(GT, tf.squeeze(pred), from_logits = True)
      grads = tape.gradient(loss, model.trainable_weights)
      optim.apply_gradients(zip(grads, model.trainable_weights))
      total_loss += loss.numpy().item()
      count += 1
    # Validation Run  
    pred = model(validation_corpus, training = False)
    logits = np.squeeze(tf.sigmoid(pred).numpy())
    # Round to 1 or 0
    ones = logits >= 0.5
    logits[:] = 0
    logits[ones] = 1
    accuracy = np.sum(logits == np_validation_GT) / np_validation_GT.shape[0]
    print(f"EPOCH: {EPOCH}, total_loss: {total_loss/ count}")
    print(f"Accuracy: {accuracy}")

In [17]:
training_fn(reviews_train, sentiments_train, reviews_val, sentiments_val, 50)


EPOCH: 0, total_loss: 0.4522684666735586
Accuracy: 0.875



EPOCH: 1, total_loss: 0.5051681346958503
Accuracy: 0.9375



EPOCH: 2, total_loss: 0.4480040474445559
Accuracy: 1.0



EPOCH: 3, total_loss: 0.4787168111070059
Accuracy: 1.0



EPOCH: 4, total_loss: 0.40639914832718205
Accuracy: 0.9375


...


EPOCH: 45, total_loss: 0.19054632032930385
Accuracy: 0.9375



EPOCH: 46, total_loss: 0.2413911386684049
Accuracy: 1.0



EPOCH: 47, total_loss: 0.18905864760745317
Accuracy: 1.0



EPOCH: 48, total_loss: 0.20356749414349906
Accuracy: 1.0



EPOCH: 49, total_loss: 0.1706838806567248
Accuracy: 1.0
